# Test for Cubo things



In [ ]:
import cubo
import xarray as xr
from sen2nbar.nbar import nbar_cubo
import spyndex
import geopandas as gpd
import numpy as np
from PIL import Image

In [ ]:
# Test region: Leipzig
#lat = 51.34648302388849, lon = 12.371403625185186,
try:
    da = cubo.create(
        lat = 51.34648302388849, lon = 12.371403625185186,
        collection = "sentinel-2-l2a",
        # Some bands are not processed by nbar_cubo, so we don't need to load them
        # bands = ["B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B09", "B11", "B12"],
        bands = ["B02", "B03", "B04", "B05", "B06", "B07", "B08", "B11", "B12"],
        start_date = "2020-01-01",
        end_date = "2020-04-30",
        units="m", # to have edge_size in meters
        edge_size = 256,
        resolution = 10,
        query = {"eo:cloud_cover": {"lt": 10}}
    )
    
    print("data found:")
    print("  size: ", da.nbytes / 1e6, "MB")
    print("  time: ", len(da.time), "steps")

except Exception as e:
    print(f"Error: {e}")
    da = None

In [ ]:
da = nbar_cubo(da)

# Leave this here for now
#da_grouped = da.groupby("time")
#da_cleaned = da_grouped.mean(dim="time", skipna=True)
# I don't know what It means anymore. 


In [ ]:
# download the data
da_loaded = da.load()

In [ ]:
# plot something for testing

(da_loaded.sel(band=["B04", "B03", "B02"])).isel(time=0).plot.imshow(robust=True)
#da_loaded.sel(band=["B04", "B03", "B02"]).plot.imshow(robust=True, col="time", col_wrap=4)

In [ ]:
important_indices = ['NDVI']

indices = spyndex.computeIndex(
    index = important_indices,
    params = {
        "B": da.sel(band = "B02"),
        "G": da.sel(band = "B03"),
        "R": da.sel(band = "B04"),
        "RE1": da.sel(band = "B05"),
        "RE2": da.sel(band = "B06"),
        "RE3": da.sel(band = "B07"),
        "N": da.sel(band = "B08"),
        "S1": da.sel(band = "B11"),
        "S2": da.sel(band = "B12")
    }
)
abs(indices[1]).plot.imshow(cmap="viridis")

In [ ]:
# to keep things here:
#tt = Image.fromarray(((da_cleaned.sel(band=["B04", "B03", "B02"]).isel(time=0)/3000).clip(0,1)*255).astype(np.uint8).transpose("y", "x", "band").values)
#tt.save("test.png")
#(da_cleaned.sel(band=["B04", "B03", "B02"]).isel(time=0)/10000).clip(0,1).plot.imshow(robust=True)